In [1]:
import os
import numpy as np

import polynomials

In [2]:
ring = polynomials.PolynomialRing(sofile=os.path.join(".", 'polynomialslib.so'))

polynomial1 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,1,0]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(1.0,[0,1,0]), (1.0,[0,0,0])])

quotients, remainder = ring.div(polynomial1, [polynomial2, polynomial3])
assert(ring.add(ring.mul(quotients[0], polynomial2), ring.mul(quotients[1], polynomial3), remainder).terms() == polynomial1.terms())

In [3]:
polynomial1 = ring.polynomial_from_terms([(1.0,[2,0,0]), (-2.0,[1,0,1]), (5.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,1,1]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(3.0,[0,2,0]), (-8.0,[1,0,1])])

groebner = ring.buchbergers(True, polynomial1, polynomial2, polynomial3)
quotients, remainder = ring.div(ring.polynomial_from_terms([(1.0,[2,0,0])]), groebner)
print quotients, remainder

[[(1.0, [2, 0, 0])]] []


In [4]:
polynomial1 = ring.polynomial_from_terms([(1.0,[2,0,0]), (-2.0,[1,0,1]), (5.0,[0,0,0])])
polynomial2 = ring.polynomial_from_terms([(1.0,[1,2,0]), (1.0,[0,1,1]), (1.0,[0,0,0])])
polynomial3 = ring.polynomial_from_terms([(3.0,[0,2,0]), (-8.0,[1,0,1])])

groebner = ring.buchbergers(False, polynomial1, polynomial2, polynomial3)
print polynomials.find_standard_monomial_basis(groebner)

set([(0, 1, 2), (1, 1, 0), (0, 0, 2), (0, 0, 3), (0, 0, 4)])
